In [156]:
import warnings

import pandas as pd
import numpy as np
import scipy.io as sio

pd.set_option('display.max_columns', None)

from cmlreaders import CMLReader, get_data_index

In [105]:
df = get_data_index('r1')
df = df[df['experiment']=='TH1']
subjects = df[['subject', 'montage', 'localization', 'experiment']].drop_duplicates()
subjects.index = range(len(subjects))

In [31]:
def subject_id(subject, montage):
    return (subject if montage == 0
                else f"{subject}_{int(montage)}")

In [35]:
def load_matlab_contacts(subject, montage):
    """ This is more consistent than cmlreaders version which breaks
        sometimes for no reason. Its also a lot more obvious what
        is going on whereas with cmlreaders its kinda a mystery :)
    """
    subj_str = subject_id(subject, montage)
    path = f'/data/eeg/{subj_str}/tal/{subj_str}_talLocs_database_monopol.mat'
    contacts = pd.DataFrame(sio.loadmat(path, squeeze_me=True)['talStruct'])
    
    if subject=='R1059J' and montage==1:
        # this contact is a duplicate of contact 114 
        contacts = contacts[contacts['channel'] != 113]
        contacts.index = range(len(contacts))
    
    return contacts

In [126]:
def load_contacts(**subject_series):
    
    reader = CMLReader(**subject_series)
    contacts = reader.load('contacts')
    
    matlab_contacts = load_matlab_contacts(
        subject_series['subject'], subject_series['montage'])
    for i in range(1,6):
        contacts[f'Loc{i}'] = matlab_contacts[f'Loc{i}']
    
    if not 'stein.region' in contacts:
        contacts['stein.region'] = np.nan
        
        if 'locTag' in matlab_contacts:
            loctags =  np.array([t if len(t)>0 else np.nan
                        for t in matlab_contacts['locTag']])
            contacts['stein.region'] = loctags

    contacts['stein.hemi'] = [np.nan if pd.isnull(t) or t == 'nan'
                              else t.split(' ')[0]
                              for t in contacts['stein.region']]
    contacts['stein.name'] = [np.nan if pd.isnull(t) or t == 'nan'
                                else ' '.join(t.split(' ')[1:])
                                for t in contacts['stein.region']]
    
    return contacts

In [127]:
warnings.filterwarnings('ignore')

In [136]:
rois = ['Left EC', 'Right EC', 'Left CA1', 'Right CA1']

roi_locs = {}
for roi in rois:
    roi_locs[roi] = []

for i, subject in subjects.iterrows():
    contacts = load_contacts(**subject)
    for roi in rois:
        if roi in contacts['stein.region'].unique():
            roi_locs[roi].append(True)
        else:
            roi_locs[roi].append(False)

for roi in rois:
    roi_locs[roi] = np.array(roi_locs[roi])


In [145]:
subjects = subjects.loc[roi_locs['Right EC']&roi_locs['Right CA1']&roi_locs['Left EC']&roi_locs['Left CA1']]
subjects.index = range(len(subjects))

In [147]:
subject = subjects.iloc[0]

In [151]:
reader = CMLReader(**subject, session=0)

In [152]:
events = reader.load('events')

In [154]:
events = events[events['type']=='CHEST']

In [162]:
recalled = events[events['recalled']]
not_recalled = events[(~events['recalled'])
                      & ~(events['item_name']=='')
                     ]
empty = events[events['item_name']=='']

In [ ]:
def load_eeg(events, channels, subject_series
            \\\\\\\\\\\\\\\):